In [ ]:
ALGORITHM = 'ac3'
PUZZLE_TYPE = 'easy'
PUZZLE_PATH = '/content/drive/MyDrive/Puzzles/Evil-P1.txt'
import os
import numpy as np
import random
import math

In [ ]:
# Access Puzzle File, Create Puzzle Array
def read_prepare_file(PUZZLE_PATH):
  #PARAMETERS
  # PUZZLE_PATH: file path to the designated puzzle, including puzzle name
  #RETURNS
  #int_puzzle_array: 9x9 int np.array puzzle that has 0 for unfilled cells and the prefilled numbers in the rest
  # int_bool_array: 9x9 bool np.array puzzle that holds True for prefilled cells and False for unfilled
  #PUZZLE_NUMBER: extracted puzzle number from the puzzle path

  #get filename and puzzle number for exporting at the end
  filename = os.path.basename(PUZZLE_PATH)
  PUZZLE_NUMBER = filename.split('-')[1].split('.')[0]

  int_puzzle_list = []
  # opens puzzle file specified in parameters and creates int_puzzle_list with given values
  #unassigned values are given the int 0
  with open(PUZZLE_PATH, 'r', encoding="utf-8-sig") as puzzle_file:
    j=0
    for line in puzzle_file:
      line = line.strip().split(",")
      i = 0
      for val in line:
        if val == '?':
          line[i] = 0
        else:
          line[i] = int(val)
        i += 1
      int_puzzle_list.append(line)
      j += 1

  #convert int_puzzle_list to numpy array
  int_puzzle_array = np.array(int_puzzle_list, dtype=int)

  #creates mirror boolean array that holds True for preassigned spots and false for unassigned spots
  bool_puzzle_array = int_puzzle_array != 0

  # makes boolean array unchangeable (hopefully)
  bool_puzzle_array.flags.writeable = False
  return int_puzzle_array, bool_puzzle_array, PUZZLE_NUMBER

#for testing
int_puzzle_array, bool_puzzle_array, PUZZLE_NUMBER = read_prepare_file(PUZZLE_PATH)


In [ ]:
# FOR SA AND GA fill empty puzzle spaces at random with numbers 1-9

def fill_puzzle(sudoku, fixed):
  # PARAMETERS
  # sudoku:9x9 np.array that is partially completed with integers 1-9 and any uncompeleted spaces=0
  #fixed: 9x9 boolean array that is True for preassigned spots and False for unassigned spots
  #RETURNS
  # filled 9x9 np.array holding integers 1-9

  filled_sudoku = sudoku.copy()
  for i in range(9):
    for j in range(9):
      if not fixed[i,j]:
        filled_sudoku[i,j] = random.randint(1,9)
  return filled_sudoku

In [ ]:
#Simulated Annealing Functions


# Function to compute conflicts, where every repeated number in a row, column or box counts as 1
# conflicts are summed
# FOR NUMBER OF CONFLICTS OVER THE ENTIRE PUZZLE
def calculate_conflicts(sudoku):
  #PARAMETERS
  #sudoku:9x9 np.array tjat holds integers 1-9
  #RETURNS
  # number of conflicts within the entire puzzle (int)
  conflicts = 0

  # by column
  for j in range(9):
    col = sudoku[:, j]
    # np.unqiue creates a list of unique numbers, meaning any numbers repeated will only be counted 1x
    # making the result of 9- (len) the number of times a repeat occurs
    conflicts += 9 - len(np.unique(col))

  # by row
  for i in range(9):
    row = sudoku[i, :]
    #same logic as above
    conflicts += 9 - len(np.unique(row))

  # by box
  for box_i in range(3):
    for box_j in range(3):
      #constructs 3x3 box
      box = sudoku[box_i*3:(box_i+1)*3, box_j*3:(box_j+1)*3].flatten()
      conflicts += 9 - len(np.unique(box))

  return conflicts

#CALCULATE THE NUMBER OF CONFLICTS CAUSED BY A SPECIFC CELL ASSIGNMENT/VALUE CHANGE
# in the cell's respective row, column and box

def calculate_cell_conflicts(sudoku, row, col, val=None):
  # PARAMETERS
  #sudoku: 9x9 np.array filled to satisfy row-wise completion
  # row: int that indicates the row of a specific value
  # col: int that indicicate the column of specific value
  # value: int, value of the cell (a way to test the conflicts of a value without changing it in the actual puzzle)
  #RETURNS
  # conflicts: an int that is the number of conflicts caused in the respecitve column, row and box of a specified val

  #locate value if not given (good for when testing possible values)
  if val == None:
    val = sudoku[row, col]
  conflicts = 0

  #by row
  for c in range(9):
    if c != col and sudoku[row,c] == val:
      conflicts += 1

  # by column
  for r in range(9):
    if r != row and sudoku[r, col] == val:
      conflicts += 1

  # by box
  box_row, box_col = 3*(row//3), 3*(col//3)
  box_vals = sudoku[box_row:box_row+3, box_col:box_col+3].flatten()
  box_vals = np.delete(box_vals, np.where(box_vals == val)) #not sure if this works
  conflicts += np.sum(box_vals == val)

  return conflicts

# Min Conflict Heuristic
# Generates possible neighbors by choosing a random unassigned cell and calculating
# the conflict for putting number 1-9 in that cell, then takes the k possible values
# with the least conflict and chooses one at random to introduce more randomization

def min_conflict_neighbor(current, fixed, k=3):
  # PARAMETERS
  #current: 9x9 np.array filled to satisfy row-wise completion
  # fixed: 9x9 boolean array that is True for preassigned spots and False for unassigned spots
  #k: int, number of top candidate swaps to consider to add randomness and expand search space
  # RETURNS
  # sudoku 9x9 np.array int with swapped elements if the swap reduces conflict or is neutral
  neighbor = current.copy()

  #create list of all unassigned cells that can be changed
  unassigned_indices = [(r,c) for r in range(9) for c in range(9) if not fixed[r,c]]

  #choose random unassigned cell and extract its current value
  row,col = random.choice(unassigned_indices)
  current_val = neighbor[row,col]

  #generate list of possible swaps
  possible_swaps = []
  for val in range (1,10):
    if val == current_val:
      continue
      #for each possible new cell assignment, calculate the conflict caused by that cell change/swap
    new_conflicts = calculate_cell_conflicts(neighbor, row, col, val)
    possible_swaps.append((val, new_conflicts))
  #sort by which changes cause the least conflict
  possible_swaps.sort(key=lambda x: x[1])
  #identify the top k changes that cause the least conflict
  top_swaps = possible_swaps[:min(k, len(possible_swaps))]
  #from top k changes select one change at random and implement it
  new_val, conf = random.choice(top_swaps)
  neighbor[row,col] = new_val

  return neighbor

# cooling schedule to define the probability by which a neighbor is selected or not
def schedule(t, T0=1, tau =1):
  # t = number of iterations
  # T0 = initial temperature
  # tau = constant by which the cooling schedule is determined
  return (T0*tau)/(tau+t)

def simulated_annealing(unfilled_sudoku, fixed, t0=1, tau=1, max_iters = 100000,
                       stall_limit = 200):
  # PARAMETERS
  # unfilled_sudoku: 9x9 np.array with assigned values and unassigned spots are 0
  # fixed: 9x9 boolean array that is True for preassigned spots and False for unassigned spots
  # t0 = initial temperature for cooling function
  # tau = constant for cooling function
  # max iters = the number of times the function is allowed to loop before termination if solution not found
  # stall_limit = the number of time a puzzle is allowed to not change to a new neighbor before being reshuffled
  # RETURNS
  # original conflicts: the number of conflicts caused by the first iteration of a filled puzzle
  # decisions: number of times a cell for a neighboring puzzle was changed and kept
  # result: output puzzle np.array holding ints 1-9

  filled = fill_puzzle(unfilled_sudoku, fixed)
  original_conflicts = calculate_conflicts(filled)
  current = filled.copy()
  current_value = -calculate_conflicts(current) #negative conflicts for maximization problem
  best = current.copy()
  best_value = current_value
  decisions = 0
  stalls = 0
# iterator and value by which cooling schedule is calculated
  for t in range(1, max_iters +1):
    T = schedule(t, t0, tau)
    # return current if T = 0
    if T == 0:
      break
    #using min conflict function, calculate new neighbor
    next_node = min_conflict_neighbor(current, fixed, k=3)
    next_value = -calculate_conflicts(next_node)
    dE = next_value - current_value
    # dE = difference in energy, or difference in puzzle quality, if it is better, accept, if worse
    # accept only with a certain probability according to the schedule
    #SA acceptance of new state or not
    if dE > 0 or random.random() < math.exp(dE/T):
      current = next_node
      current_value = next_value
      decisions += 1
      stalls = 0 #reset if the puzzle is changed
    else:
      stalls += 1
      # if puzzle is not changed, count as a stall

    #update global bests if changed
    if current_value > best_value:
      best = current.copy()
      best_value = current_value

    #stop if solution is found:
    if best_value == 0:
      break
    #if too many stalls then shuffle puzzle to escape local optimum
    if stalls >= stall_limit:
      current = fill_puzzle(current, fixed)
      current_value = -calculate_conflicts(current)
      stalls = 0

  return original_conflicts, best, decisions

#function to automate puzzle import, algorithm to solve puzzle, printing of puzzle results and puzzle export
def exectute_SA(int_puzzle_array, bool_puzzle_array, GROUP_ID, PUZZLE_TYPE, PUZZLE_NUMBER, ALGORITHM):
  original_conflicts, result, decisions = simulated_annealing(int_puzzle_array, bool_puzzle_array)
  print("Original Conflicts:"+str(original_conflicts))
  print("Output Puzzle:")
  print(result)
  print("Output Conflicts:"+ str(calculate_conflicts(result)))
  print("Decisions:"+str(decisions))
  export_puzzle(result, GROUP_ID, ALGORITHM, PUZZLE_TYPE, PUZZLE_NUMBER)

In [ ]:
# Genetic Algorithm Functions

# Tournament Selection
def tourney_selection(population, k=2):
  #PARAMETERS
  #population: array of 9x9 int np.array sudoku puzzles
  # k: int, tournament size
  # RETURNS
  # "player" from tournament with the fewest conflicts
  players = random.sample(population, k)
  return min(players, key=calculate_conflicts)

#Crossover
# Recombination of two parents to create a child via row-based crossover
def crossover(parent1, parent2, fixed):
  #PARAMETERS
  #parent1: 9x9 int np.array sudoku puzzle
  #parent2: 9x9 int np.array sudoku puzzle
  #fixed: 9x9 boolean array that is True for preassigned spots and False for unassigned spots
  # RETURNS
  # child: new 9x9 int puzzle array created from parents
  child = parent1.copy()
  # for each row replace with a 50% chance the row with the same row from parent2
  for row in range(9):
    if random.random() < 0.5:
        # replace an entire row in parent 1 with parent 2 row
        for col in range(9):
          if not fixed[row, col]:
            child[row, col] = parent2[row, col]
  return child


#Mutation: changing non fixed cells to a random value between 1 and 9 at a probability
# defined by the mutation rate
def mutate(sudoku, fixed, decisions, mutation_rate = 0.1):
  #PARAMETERS
  #sudoku: 9x9 int np.array sudoku puzzle
  #fixed: 9x9 boolean array that is True for preassigned spots and False for unassigned spots
  #mutation_rate: float, probability of that a change in a cell will be made
  #decisions : current decision count
  #RETURNS
  # child: 9x9 int np.array sudoku puzzle
  # decisions: new decision count
  child = sudoku.copy()
  #for unassigned cell in a sudoku puzzle at a probability given by mutation rate change the cell with a random value
  # between 1 and 9
  for row in range(9):
    for col in range(9):
      if not fixed[row,col] and random.random() < mutation_rate:
        child[row,col] = random.randint(1,9)
        decisions += 1
  return child, decisions

# Genetic Algorithm implementation with k-elitism
def genetic_algorithm(sudoku, fixed, pop_size = 50, max_gens = 5000, k=2,
                      mutation_rate = 0.1, crossover_rate = 0.95):
  #PARAMETERS
  #sudoku: 9x9 int np.array sudoku puzzle
  #fixed: 9x9 boolean array that is True for preassigned spots and False for unassigned spots
  #pop_size: int, size of population
  # max_gens: int, maximum number of generations allowed to be generate by the function
  # k: int, tournament size for tournament selection
  #mutation_rate: float, probability of that a change in a cell will be made
  #crossover_rate: the probability of which a child will inherit from parent 1 of parent 2
  #RETURN
  # original conflicts: the number of conflicts in the initially filled puzzle
  # decisions: the number of times the function changed a number in a cell that
  # was passed on to a new generation
  # best: the output puzzle, with the smallest number of conflicts found when the iterations ended or the solution was found

  #create intial sudoku by which to compare the change in conflicts
  initial_sudoku = fill_puzzle(sudoku, fixed)
  initial_conflicts = calculate_conflicts(initial_sudoku)
  #create initial population
  population = [fill_puzzle(sudoku.copy(), fixed) for i in range(pop_size-1)]
  population.append(initial_sudoku)
  #caclulate fitness of intial population and identify the current best and its index
  population_fitness =[calculate_conflicts(ind) for ind in population]
  best_index = np.argmin(population_fitness)
  best = population[best_index].copy()
  best_score = population_fitness[best_index]
  decisions = 0

#iterator-- number of generations that will be created before function terminates
  for gen in range(max_gens):
    #create new generation to replace old
    new_population = []

    #generate enough children to fill new population of same size as old
    while len(new_population) < pop_size:
      #selection
      parent1 = tourney_selection(population, k)
      parent2 = tourney_selection(population, k)
      #crossover that will occur if the random value (0-1) generated is below the crossover rate
      if random.random() < crossover_rate:
        child = crossover(parent1, parent2, fixed)
      else:
        child = parent1.copy()
      #mutation
      child, decisions = mutate(child, fixed, decisions, mutation_rate)
      #add child to new generation's pop
      new_population.append(child)

    #get fitness for new population
    new_fitness = [calculate_conflicts(ind) for ind in new_population]

    #find n best puzzles from old generation
    n_elites = 5
    elite_indices = np.argsort(population_fitness)[:n_elites]
    elites = [population[i].copy() for i in elite_indices]

    # elitism- preserve top n elites from old population into new population
    worst_indices = np.argsort(new_fitness)[-n_elites:]
    for elite, idx in zip(elites, worst_indices):
      new_population[idx] = elite.copy()
      new_fitness[idx] = calculate_conflicts(elite)

    # update population
    population = new_population
    population_fitness = new_fitness

    #update current best
    current_best_index = np.argmin(population_fitness)
    current_best_score = population_fitness[current_best_index]
    #update all-time best
    if current_best_score < best_score:
          best = population[current_best_index].copy()
          best_score = current_best_score
  #if solution found return function
    if best_score == 0:
      break

  return initial_conflicts, best, decisions

#function to string together intializing the puzzle, algorithm execution and final puzzle export
def execute_GA(int_puzzle_array, bool_puzzle_array, GROUP_ID, PUZZLE_TYPE, PUZZLE_NUMBER, ALGORITHM):
  original_conflicts, result, decisions = genetic_algorithm(int_puzzle_array, bool_puzzle_array)
  print("Original Conflicts:"+str(original_conflicts))
  print("Output Puzzle:")
  print(result)
  print("Output Conflicts:"+ str(calculate_conflicts(result)))
  print("Decisions:"+str(decisions))
  export_puzzle(result, GROUP_ID, ALGORITHM, PUZZLE_TYPE, PUZZLE_NUMBER)

In [ ]:
# Verify possible number is legal
def is_legal(sudoku_board, row, col, num):
  # PARAMETERS
  # sudoku_board: 9x9 np.array representing current state of sudoku board
  # row: int row index
  # col: int column index
  # num: int number (1-9) being placed in cell

  # If number is already in row of selected cell number is not legal
  if num in sudoku_board[row]:
    return False
  # If number is already in column of selected cell number is not legal
  if num in sudoku_board[:, col]:
    return False
  # If number is already in 3x3 grid of selected cell number is not legal
  r_start = 3*(row//3)
  c_start = 3*(col//3)
  for r in range(r_start, r_start+3):
    for c in range(c_start, c_start+3):
      if sudoku_board[r][c] == num:
        return False
  # number is legal
  return True

In [ ]:
# Simple backtracking
def simple_backtracking(int_puzzle_array, bool_puzzle_array, counter=None):
  # PARAMETERS
  # int_puzzle_array: 9x9 np.array representing current state of sudoku board
  # bool_puzzle_array: 9x9 boolean array that is True for preassigned spots and False for unassigned spots
  # counter: counts total number of cell changes

  if counter is None:
        counter = {"decisions": 0} # count of cell changes

  for row in range(9):
    for col in range(9):
      if int_puzzle_array[row][col] == 0 and not bool_puzzle_array[row][col]:
        for num in range(1,10):
          if is_legal(int_puzzle_array, row, col, num):
            int_puzzle_array[row][col] = num
            counter["decisions"] += 1
            if simple_backtracking(int_puzzle_array, bool_puzzle_array, counter):
              return True # recursively solved
            int_puzzle_array[row][col] = 0
            counter["decisions"] += 1
        return False # trigger backtrack (no valid # found)
  return True # puzzle is solved if no empty cells


In [ ]:
import copy
# Backtracking with forward checking

def forward_checking_backtracking(int_puzzle_array, bool_puzzle_array, counter=None):
  # PARAMETERS
  # int_puzzle_array: 9x9 np.array representing current state of sudoku board
  # bool_puzzle_array: 9x9 boolean array that is True for preassigned spots and False for unassigned spots
  # counter: counts total number of cell changes

  if counter is None:
        counter = {"decisions": 0} # count of cell changes

  # Find first empty cell
  def find_empty_cell():
    for row in range(9):
      for col in range(9):
        if int_puzzle_array[row][col] == 0 and not bool_puzzle_array[row][col]:
          return row, col
    return None

  def update_domain(domains, row, col, num):
    domain_new = copy.deepcopy(domains)
    for i in range(9):
      domain_new[row][i].discard(num)
      domain_new[i][col].discard(num)
    r_start, c_start = 3*(row//3), 3*(col//3)
    for r in range(r_start, r_start+3):
        for c in range(c_start, c_start+3):
            domain_new[r][c].discard(num)
    domain_new[row][col] = {num}
    return domain_new

  # Create domains
  domains = []
  for r in range(9):
    row_domains = []
    for c in range(9):
      if bool_puzzle_array[r][c]:
        row_domains.append({int_puzzle_array[r][c]})
      else:
        legal_numbers = {n for n in range(1, 10) if is_legal(int_puzzle_array, r, c, n)}
        row_domains.append(legal_numbers)
    domains.append(row_domains)

  def solve_recursively(domains):
    cell = find_empty_cell()
    if not cell:
      return True # puzzle is solved

    row, col = cell

    for num in sorted(domains[row][col]):
      if is_legal(int_puzzle_array, row, col, num):
        int_puzzle_array[row][col] = num
        counter["decisions"] += 1

        domain_new = update_domain(domains, row, col, num)

        # Backtrack if no empty domain in constraint neighbors
        if all(domain_new[r][c] for r in range(9) for c in range(9) if not bool_puzzle_array[r][c]):
          if solve_recursively(domain_new):
            return True
        int_puzzle_array[row][col] = 0 # remove assignment
        counter["decisions"] += 1

    return False # trigger backtrack

  if solve_recursively(domains): # puzzle is solved if no empty cells
    return int_puzzle_array
  else:
    return None

In [ ]:
import copy

# Backtracking with arc consistency

def arc_consistency_backtracking(int_puzzle_array, bool_puzzle_array, counter=None):
  # PARAMETERS
  # int_puzzle_array: 9x9 np.array representing current state of sudoku board
  # bool_puzzle_array: 9x9 boolean array that is True for preassigned spots and False for unassigned spots
  # counter: counts total number of cell changes

  # Similar process to backtracking with forward checking, but propogate domains forward using arc consistency
  # If a neighbors domain changes, update its neighbors domains, and so on

  if counter is None:
        counter = {"decisions": 0} # count of cell changes

  # Find first empty cell
  def find_empty_cell():
    for row in range(9):
      for col in range(9):
        if int_puzzle_array[row][col] == 0 and not bool_puzzle_array[row][col]:
          return row, col
    return None

  def arc_neighbors(row, col):
    neighbors = []
    for i in range(9):
      neighbors.append((row,i))
      neighbors.append((i,col))
    r_start, c_start = 3*(row//3), 3*(col//3)
    for r in range(r_start, r_start+3):
        for c in range(c_start, c_start+3):
          neighbors.append((r,c))
    neighbors = list(set(neighbors))
    neighbors.remove((row, col))
    return neighbors

  def arc_consistency(domains):
    changed = True
    while changed:
      changed = False
      for x_r in range(9):
        for x_c in range(9):
          for (y_r, y_c) in arc_neighbors(x_r, x_c):
            if change(domains, (x_r, x_c), (y_r, y_c)):
              changed = True
              if not domains[x_r][x_c]:
                return False
    return True

  def change(domains, x, y):
    x_r, x_c = x
    y_r, y_c = y
    removed = False
    remove = []
    for val in domains[x_r][x_c]:
      supported = False
      for val_2 in domains[y_r][y_c]:
        if val_2 != val:
          supported = True
          break
      if not supported:
        remove.append(val)

    for val in remove:
      domains[x_r][x_c].remove(val)
      removed = True
      if counter is not None:
        counter["decisions"] += 1

    return removed

  # Create domains
  domains = []
  for r in range(9):
    row_domains = []
    for c in range(9):
      if bool_puzzle_array[r][c]:
        row_domains.append([int_puzzle_array[r][c]])
      else:
        legal_numbers = [n for n in range(1, 10) if is_legal(int_puzzle_array, r, c, n)]
        row_domains.append(legal_numbers)
    domains.append(row_domains)

  def solve_recursively(domains):
    cell = find_empty_cell()
    if not cell:
      return True # puzzle is solved

    row, col = cell

    for num in sorted(domains[row][col]):
      if is_legal(int_puzzle_array, row, col, num):
        int_puzzle_array[row][col] = num
        counter["decisions"] += 1

        domain_new = copy.deepcopy(domains)
        domain_new[row][col] = [num]

        if arc_consistency(domain_new):
          if solve_recursively(domain_new):
            return True

        int_puzzle_array[row][col] = 0
        counter["decisions"] += 1

    return False

  if solve_recursively(domains): # puzzle is solved if no empty cells
    return int_puzzle_array
  else:
    return None

In [ ]:
# Export Puzzle
def export_puzzle(int_puzzle_array,  ALGORITHM, PUZZLE_TYPE, PUZZLE_NUMBER):
  file_name = f"{ALGORITHM}_{PUZZLE_TYPE}_{PUZZLE_NUMBER}.txt"
  save_path = os.path.join(os.getcwd(), file_name)
  print(save_path)

  with open(file_name, "w", encoding="utf-8-sig") as file:
    for r in range(int_puzzle_array.shape[0]):
      row = []
      for c in range(int_puzzle_array.shape[1]):
        val = int_puzzle_array[r,c]
        row.append(str(val))
      file.write(",".join(row) + "\n")

In [ ]:
#FOR EXECUTION, RUN THIS CELL

#load in puzzle, get arrays and puzzle number
int_puzzle_array, bool_puzzle_array, PUZZLE_NUMBER = read_prepare_file(PUZZLE_PATH)

# based on algorithm parameter, proceed
if(ALGORITHM == 'bt'):
  counter = {"decisions": 0}
  simple_backtracking(int_puzzle_array, bool_puzzle_array, counter)
  export_puzzle(int_puzzle_array, ALGORITHM, PUZZLE_TYPE, PUZZLE_NUMBER)

elif(ALGORITHM == 'fc'):
  counter = {"decisions": 0}
  forward_checking_backtracking(int_puzzle_array, bool_puzzle_array, counter)
  export_puzzle(int_puzzle_array, ALGORITHM, PUZZLE_TYPE, PUZZLE_NUMBER)

elif (ALGORITHM == 'ac3'):
  counter = {"decisions": 0}
  arc_consistency_backtracking(int_puzzle_array, bool_puzzle_array, counter)
  export_puzzle(int_puzzle_array,  ALGORITHM, PUZZLE_TYPE, PUZZLE_NUMBER)

#SIMULATED ANNEALING
elif (ALGORITHM == 'sa'):
  exectute_SA(int_puzzle_array, bool_puzzle_array, PUZZLE_TYPE, PUZZLE_NUMBER, ALGORITHM)
#GENETIC ALGORITHM
elif (ALGORITHM == 'ga'):
  execute_GA(int_puzzle_array, bool_puzzle_array, PUZZLE_TYPE, PUZZLE_NUMBER, ALGORITHM)
#INVLAID ALGORITHM
else:
  print("Invalid Algorithm")

/content/Group11_ac3_easy_P1.txt
